In [48]:
from FundData import *
from helper import Helper
import pprint, json
import fitz

dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
#dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"
#fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'

dry_path = r'\output\DryRun.pdf'
fin_path = r'\files\financial_indices.xlsx'


mutual_fund = Helper.get_fund_paths(fund_path)

In [2]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list

def extract_spans(data):
    final_span = []
    for pgn,blocks in enumerate(data):
        spans = []
        print(f"___{pgn}___")
        for num,block in enumerate(blocks.get('blocks',[])):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if num in range(3,10):
                        print(span['text'], span['size'])
        final_span.append(spans)
    return final_span

def get_proper_fund_names(path:str,pages:list):
        
        doc = fitz.open(path)
        final_fund_names = dict()
        
        for pgn in range(doc.page_count):
            fund_names = ''
            if pgn in pages:
                page = doc[pgn]            
                blocks = page.get_text("dict")['blocks']
                
                sorted_blocks = sorted(blocks,key=lambda k:(k['bbox'][1],k['bbox'][0]))
                for count,block in enumerate(sorted_blocks):
                    for line in block.get("lines", []):
                        for span in line.get("spans", []):
                            text = span['text'].strip()
                            if count in range(0,2): #contains fund name        
                                fund_names += f'{text} '
            if matches:= re.search(r'\bMahindra.*?(Fund|ETF|EOF|FOF|FTF|Path)\b', fund_names, re.IGNORECASE):           
                final_fund_names[pgn] = matches.group()
            else:
                final_fund_names[pgn] = ''

        return final_fund_names

In [56]:
# Lines to be drawn (start and end points)
lines = [
    ((220, 0), (220, 812)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0, 5, 300, 812)]

sample_path  = mutual_fund['Zerodha Mutual Fund']
pages = [i for i in range(5,55)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dir_path +dry_path)
data = Helper.get_all_pdf_data(sample_path)

Modified PDF saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf


In [43]:
def get_all_pdf_data(path:str, pages:list, clip:set):
    
        doc = fitz.open(path)
        for pgn in pages:
            page = doc[pgn]
            blocks = page.get_text('dict', clip=clip)['blocks']
            combined_text = []
    
            for block in blocks:
                for line in block.get('lines', []):
                    for span in line.get('spans', []):
                        text = span.get('text', "")
                        if re.search(r'^Baroda BNP', text) or re.search(r'Fund$', text):
                            combined_text.append(text)
            
            print(f"\n----{pgn}----"," ".join(combined_text))

        doc.close()

In [5]:
clipped_data = Helper.get_clipped_data(sample_path,[(0,0,210,300)])

In [63]:
def get_proper_fund_names(path:str, pages:list, bbox:set):
        doc = fitz.open(path)
        
        fund_titles = dict()
        for pgn in range(doc.page_count):
            if pgn in pages:
                page = doc[pgn]
                blocks = page.get_text('dict', clip=bbox)['blocks']
                combined_text = []
        
                for block in blocks:
                    for line in block.get('lines', []):
                        for span in line.get('spans', []):
                            text = span.get('text', "")
                            if re.search(r'^Zerodha', text) or re.search(r'(Fund|ETF|FOF|EOF|FTF|Path)$', text):
                                combined_text.append(text)
                fund_titles[pgn] = " ".join(combined_text)
            else:
                fund_titles[pgn] = ""

In [58]:
data[5]

{'pgn': 5,
 'blocks': [{'number': 0,
   'type': 0,
   'bbox': (98.0, 101.0, 388.1028747558594, 155.0),
   'lines': [{'spans': [{'size': 24.0,
       'flags': 0,
       'font': 'Unnamed-T3',
       'color': -16777216,
       'ascender': 0.8999999761581421,
       'descender': -0.10000000149011612,
       'text': 'Zerodha ELSS Tax Saver Nifty ',
       'origin': (98.0, 101.0),
       'bbox': (98.0, 101.0, 388.1028747558594, 125.0)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (98.0, 101.0, 388.1028747558594, 125.0)},
    {'spans': [{'size': 24.0,
       'flags': 0,
       'font': 'Unnamed-T3',
       'color': -16777216,
       'ascender': 0.8999999761581421,
       'descender': -0.10000000149011612,
       'text': 'LargeMidcap 250 Index Fund',
       'origin': (98.0, 131.0),
       'bbox': (98.0, 131.0, 378.4865417480469, 155.0)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (98.0, 131.0, 378.4865417480469, 155.0)}]},
  {'number': 17,
   'type': 0,
   'bbox': (40.0, 